# Demo for Hotwired 7, May 2024

Rubin Science Platform deployment: data.lsst.cloud <br>
LSST Science Pipelines version: Weekly 2024_16 <br>
Last verified to run: Tue May 7 2024 <br>
Contact author: Melissa Graham <br>

<br>

>**Warning:** The following demonstration is extremely light on narrative descriptions
>and is intended for supervised execution only.

Learn more about the Data Preview 0 data set (e.g., image types, catalog tables)
and the LSST Science Pipelines and Rubin Science Platform functionality using the
documentation and resources available at <a href="https://dp0.lsst.io/">dp0.lsst.io</a>.

<br>


## Introduction

**Scenario**: a time-domain target of interest was identified by an alert broker,
and now additional information beyond what is in the alert packet is desired.

Alert packets only contain small (6"x6") image stamps, have histories
of only 12 months, and list only the identifiers for nearby objects.

This demo shows how to:
1. Explore the r-band deeply coadded image at the target's location.
2. Display the full historical multi-band forced photometry light curve.
3. Get additional information about potential host galaxies.

Import packages.

In [ ]:
from lsst.rsp import get_tap_service
import lsst.afw.display as afwDisplay
from lsst.daf.butler import Butler
import lsst.geom

import numpy as np
import matplotlib.pyplot as plt
from astropy.coordinates import SkyCoord

## Demo 1: Explore the r-band deeply coadded image at the target's location.

Instantiate the data butler.

In [ ]:
butler = Butler('dp02', collections='2.2i/runs/DP0.2')

Define the target coordinates in decimal degrees.

In [ ]:
ra = 67.4579
dec = -44.0802

The LSST deeply coadded images are divided up into tracts, and tracts into patches.

Use the defined `skyMap` to obtain the tract and patch of the target coordinates.

In [ ]:
skymap = butler.get('skyMap')
point = lsst.geom.SpherePoint(ra*lsst.geom.degrees,
                              dec*lsst.geom.degrees)
findTract = skymap.findTract(point)
tract = findTract.tract_id
patch = findTract.findPatch(point).getSequentialIndex()
del skymap, findTract

Use the tract, patch, and filter of interest (r) to define the `dataId` to pass to the butler.

Retrieve the deeply coadded image patch containing the target's coordinates.

In [ ]:
dataId = {'band': 'r', 'tract': tract, 'patch': patch}
image = butler.get('deepCoadd', dataId=dataId)

Set the display backend to Firefly.

In [ ]:
afwDisplay.setDefaultBackend('firefly')
afw_display = afwDisplay.Display(frame=1)

Click and drag the Firefly tab down and to the right, for split-screen view.

Display the image.

In [ ]:
afw_display.mtv(image)

Set all mask planes to transparent to see only pixel data.

In [ ]:
afw_display.setMaskTransparency(100)

Notice the displayed crosshairs are in the image center, not at the target coordinates.

Mark the target location.

Get the image WCS, transform the sky coordinates to pixel coordinate, and use a blue circle.

In [ ]:
image_wcs = image.wcs
point_pix = image_wcs.skyToPixel(point)
afw_display.dot('o', point_pix.getX(), point_pix.getY(),
                size=20, ctype='blue')

In the image, zoom in on the target. 

Play with the Firefly interface to change scaling, colormap, draw compass, make line cut plots, etc.

Close the Firefly window.

Clean up.

In [ ]:
del butler, ra, dec, point, tract, patch
del dataId, image, afw_display
del image_wcs, point_pix

## Demo #2: Display the full historical multi-band forced photometry light curve.

Instantiate the TAP service.

In [ ]:
tap_service = get_tap_service("tap")

Define colors and symbols to represent the LSST filters.

In [ ]:
filt_nms = ['u', 'g', 'r', 'i', 'z', 'y']
filt_clr = {'u': '#56b4e9', 'g': '#008060', 'r': '#ff4000',
            'i': '#850000', 'z': '#6600cc', 'y': '#000000'}
filt_sym = {'u': 'o', 'g': '^', 'r': 'v', 'i': 's', 'z': '*', 'y': 'p'}

Create the query to retrieve all of the difference-image forced photometry available
within 2 arcseconds of the target's location.

Join with the `CcdVisit` table to obtain the time of the observation, `expMidptMJD`.

In [ ]:
str_ra = '67.4579'
str_dec = '-44.0802'

query = "SELECT fsodo.band, fsodo.psfDiffFlux, fsodo.psfDiffFluxErr, "\
        "       fsodo.ccdVisitId, fsodo.diaObjectId, cv.expMidptMJD "\
        "FROM dp02_dc2_catalogs.ForcedSourceOnDiaObject AS fsodo "\
        "JOIN dp02_dc2_catalogs.CcdVisit AS cv ON cv.ccdVisitId = fsodo.ccdVisitId "\
        "WHERE CONTAINS(POINT('ICRS', coord_ra, coord_dec),"\
        "CIRCLE('ICRS'," + str_ra + ", " + str_dec + ", 0.00056))=1"

Submit the query to the TAP service.

Run it asynchronously, wait for completion.

In [ ]:
job = tap_service.submit_job(query)
job.run()
job.wait(phases=['COMPLETED', 'ERROR'])
print('Job phase is', job.phase)

Retrieve the results.

In [ ]:
tap_results = job.fetch_result().to_table()

How many unique `diaObjectId` were returned within 2 arcsec?

In [ ]:
values = np.unique(tap_results['diaObjectId'])
print(len(values))
del values

Display the full historical multi-band forced photometry light curve for this one `diaObject`.

In [ ]:
fig = plt.figure(figsize=(6, 4))
for f, filt in enumerate(filt_nms):
    fx = np.where(tap_results['band'] == filt)[0]
    plt.plot(tap_results['expMidptMJD'][fx], tap_results['psfDiffFlux'][fx], 
             filt_sym[filt], ms=4, mew=0, color=filt_clr[filt], label=filt)
    del fx
plt.xlabel('MJD')
plt.ylabel('Difference Flux [nJy]')
plt.legend(loc='upper left', ncol=2)
plt.show()

Plot the forced source light curve only for observations with
positive difference-image fluxes that have a signal-to-noise ratio > 5.

Convert from nJy to magnitudes.

In [ ]:
fig = plt.figure(figsize=(6, 4))
for f, filt in enumerate(filt_nms):
    fx = np.where((tap_results['band'] == filt) & 
                  (tap_results['psfDiffFlux'] > 5.0 * tap_results['psfDiffFluxErr']))[0]
    days = tap_results['expMidptMJD'][fx]
    mags = -2.5 * np.log10(tap_results['psfDiffFlux'][fx]) + 31.4
    plt.plot(days, mags, filt_sym[filt], ms=6, mew=0, alpha=0.7,
             color=filt_clr[filt], label=filt)
    del fx, days, mags
plt.xlabel('MJD')
plt.ylabel('Difference Magnitude [mag]')
plt.legend(loc='upper right', ncol=2)
plt.gca().invert_yaxis()
plt.show()

In the figure above, notice that the DP0.2 simulation had instances of multiple exposures
being obtained in the same filter in a given night.

This might not be the case with the final survey strategy.

Clean up.

In [ ]:
del tap_service
del filt_nms, filt_clr, filt_sym
del str_ra, str_dec
del query, job, tap_results

## Demo #3: Get additional information about potential host galaxies.

Instantiate the TAP service.

In [ ]:
tap_service = get_tap_service("tap")

The scenario is that a broker has identified a `diaObject` of interest.

The data defined in the following cell would have come from the alert packet:
the `diaObjectId`, the coordinates, and the 
`ObjectId` for the three nearest galaxies (by 2D sky separation).

In [ ]:
diao_id = 1568026726510894110
diao_coord = [63.6025914, -38.634654]
diao_objIds = [1568026726510919266, 1568026726510919261, 1568026726510919497]

Create the query for additional information from the `Object` catalog.

For this demo, object shape and magnitude in g, r, and i are returned.
In the future, additional information such as photometric redshift could be returned.

In [ ]:
list_objIds = "(" + ','.join(['%20i' % num for num in diao_objIds]) + ")"

query = "SELECT objectId, coord_ra, coord_dec, refExtendedness, "\
        "shape_xx, shape_xy, shape_yy, "\
        "scisql_nanojanskyToAbMag(g_cModelFlux) AS g_cModelMag, "\
        "scisql_nanojanskyToAbMag(r_cModelFlux) AS r_cModelMag, "\
        "scisql_nanojanskyToAbMag(i_cModelFlux) AS i_cModelMag "\
        "FROM dp02_dc2_catalogs.Object "\
        "WHERE objectId IN " + list_objIds
del list_objIds

Submit the query to the TAP service.

In [ ]:
job = tap_service.submit_job(query)
job.run()
job.wait(phases=['COMPLETED', 'ERROR'])
print('Job phase is', job.phase)

Retrieve the results.

In [ ]:
tap_results = job.fetch_result().to_table()

Option to display the table.

In [ ]:
# tap_results

Calculate the 2D sky separation (`2dsep`) and the offset in elliptical radii (`ellrad`) between the
`diaObject` of interest from the alert, and each of the three nearest extended objects.

See <a href="https://sextractor.readthedocs.io/en/latest/Position.html#ellipse-parameters-cxx-cyy-cxy">this description and graphic of the ellipse parameters CXX, CYY, CXY in the Source Extractor documentation by E. Bertin</a>.

In [ ]:
target_coord = SkyCoord(diao_coord[0], diao_coord[1], unit='deg')

tap_results['2dsep'] = np.zeros(3, dtype='float')
tap_results['ellrad'] = np.zeros(3, dtype='float')

for i in range(3):
    objra = tap_results['coord_ra'][i]
    objdec = tap_results['coord_dec'][i]
    objcoord = SkyCoord(objra, objdec, unit='deg')
    del objra, objdec
    
    temp = objcoord.separation(target_coord)
    tap_results['2dsep'][i] = temp.arcsec
    del temp
    
    temp = objcoord.spherical_offsets_to(target_coord)
    xr = 3600.0 * temp[0].deg
    yr = 3600.0 * temp[1].deg
    del temp, objcoord
    
    Ixx = tap_results['shape_xx'][i]
    Iyy = tap_results['shape_yy'][i]
    Ixy = tap_results['shape_xy'][i]
    Cxx = Iyy / ((Ixx * Iyy) - Ixy)
    Cyy = Ixx / ((Ixx * Iyy) - Ixy)
    Cxy = -2.0 * (Ixy) / ((Ixx * Iyy) - Ixy)
    tap_results['ellrad'][i] = np.sqrt((Cxx * xr**2) + (Cyy * yr**2) + (Cxy * xr * yr))

    del Ixx, Iyy, Ixy, Cxx, Cyy, Cxy

del target_coord

In [ ]:
tap_results

Which is the most nearby galaxy by 2D sky separation?

Which is the most nearby galaxy by elliptical radii?

In [ ]:
m1x = np.argmin(tap_results['2dsep'])
m2x = np.argmin(tap_results['ellrad'])
print('Nearest galaxy and its r-band magnitude and shape_xx parameter.')
print('2D separation:    %1i %5.2f %5.2f' % (m1x, tap_results['r_cModelMag'][m1x], tap_results['shape_xx'][m1x]))
print('elliptical radii: %1i %5.2f %5.2f' % (m2x, tap_results['r_cModelMag'][m2x], tap_results['shape_xx'][m2x]))
del m1x, m2x

Oh very interesting!

So there is a fainter, smaller galaxy closer to the target in 2D sky distance -- background interloper?

And there is a brighter, larger galaxy that is further from the target in 2D sky distance but closer in elliptical radii -- the potential host galaxy?

Here are some other `diaObjects` to explore.

In [ ]:
# diao_id = 1569909090417642499
# diao_coord = [69.9257038, -38.1424959]
# diao_objIds = [1569425305301455007, 1569425305301455003, 1569425305301455014]

In [ ]:
# diao_id = 1653700672547196623
# diao_coord = [70.8210894, -35.9915118]
# diao_objIds = [1653700672547231391, 1653700672547231402, 1653700672547231397]

In [ ]:
# diao_id = 1734140943235288573
# diao_coord = [52.5432991, -34.9028848]
# diao_objIds = [1734140943235326493, 1734140943235293084, 1734140943235326492]

In [ ]:
# diao_id = 1825796232526695593
# diao_coord = [71.7356252, -34.2191764]
# diao_objIds = [1739084347513803559, 1739084347513803574, 1739084347513803571]

Clean up.

In [ ]:
del tap_service
del diao_id, diao_coord, diao_objIds
del query, job, tap_results